<a href="https://colab.research.google.com/github/vishnub0/AWS-WEP/blob/main/DT%2BNB_AWS15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [2]:
import pandas as pd
import numpy as np
import boto3
import sagemaker

In [3]:
# Set SageMaker and S3 client variables
sess = sagemaker. Session()
region = sess. boto_region_name
s3_client = boto3.client("s3", region_name=region)
sagemaker_role = sagemaker.get_execution_role()

#Data Preprocessing (make sure you upload the CSV files)

In [4]:
BALANCED_DATASET = 's3://wep-team15-bucket/diabetes/diabetes_binary_5050split_health_indicators_BRFSS2015.csv'
df = pd.read_csv(BALANCED_DATASET)
df

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:294: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70687,1.0,0.0,1.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,0.0,0.0,6.0,4.0,1.0
70688,1.0,0.0,1.0,1.0,29.0,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,2.0,0.0,0.0,1.0,1.0,10.0,3.0,6.0
70689,1.0,1.0,1.0,1.0,25.0,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,5.0,15.0,0.0,1.0,0.0,13.0,6.0,4.0
70690,1.0,1.0,1.0,1.0,18.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,4.0,0.0,0.0,1.0,0.0,11.0,2.0,4.0


Normalize columns

In [5]:
df_norm = df.copy()
features_to_normalize = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth', 'Age', 'Education', 'Income']
for feature in features_to_normalize:
  max = df_norm[feature].max()
  min = df_norm[feature].min()
  df_norm[feature] = (df_norm[feature] - min) / (max - min)

In [6]:
df_norm

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,0.162791,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.50,0.166667,1.000000,0.0,1.0,0.250000,1.0,1.000000
1,0.0,1.0,1.0,1.0,0.162791,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,0.50,0.000000,0.000000,0.0,1.0,0.916667,1.0,1.000000
2,0.0,0.0,0.0,1.0,0.162791,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.00,0.000000,0.333333,0.0,1.0,1.000000,1.0,1.000000
3,0.0,1.0,1.0,1.0,0.186047,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.50,0.000000,0.100000,0.0,1.0,0.833333,1.0,1.000000
4,0.0,0.0,0.0,1.0,0.197674,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,0.25,0.000000,0.000000,0.0,0.0,0.583333,0.8,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70687,1.0,0.0,1.0,1.0,0.290698,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.75,0.000000,0.000000,0.0,0.0,0.416667,0.6,0.000000
70688,1.0,0.0,1.0,1.0,0.197674,1.0,0.0,1.0,0.0,1.0,...,1.0,0.0,0.25,0.000000,0.000000,1.0,1.0,0.750000,0.4,0.714286
70689,1.0,1.0,1.0,1.0,0.151163,0.0,0.0,1.0,0.0,1.0,...,1.0,0.0,1.00,0.500000,0.000000,1.0,0.0,1.000000,1.0,0.428571
70690,1.0,1.0,1.0,1.0,0.069767,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.75,0.000000,0.000000,1.0,0.0,0.833333,0.2,0.428571


#Feature tuning (removing columns)

In [7]:
cols_to_remove = ['Smoker', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost',
                  'MentHlth', 'Sex', 'Stroke', 'PhysActivity', 'Income', 'Education']
df_norm = df_norm.drop(cols_to_remove, axis='columns')
df_norm

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,HeartDiseaseorAttack,GenHlth,PhysHlth,DiffWalk,Age
0,0.0,1.0,0.0,1.0,0.162791,0.0,0.50,1.000000,0.0,0.250000
1,0.0,1.0,1.0,1.0,0.162791,0.0,0.50,0.000000,0.0,0.916667
2,0.0,0.0,0.0,1.0,0.162791,0.0,0.00,0.333333,0.0,1.000000
3,0.0,1.0,1.0,1.0,0.186047,0.0,0.50,0.100000,0.0,0.833333
4,0.0,0.0,0.0,1.0,0.197674,0.0,0.25,0.000000,0.0,0.583333
...,...,...,...,...,...,...,...,...,...,...
70687,1.0,0.0,1.0,1.0,0.290698,0.0,0.75,0.000000,0.0,0.416667
70688,1.0,0.0,1.0,1.0,0.197674,1.0,0.25,0.000000,1.0,0.750000
70689,1.0,1.0,1.0,1.0,0.151163,1.0,1.00,0.000000,1.0,1.000000
70690,1.0,1.0,1.0,1.0,0.069767,0.0,0.75,0.000000,1.0,0.833333


#Splitting data

In [8]:
X = df_norm.drop('Diabetes_binary', axis='columns')
y = df_norm['Diabetes_binary']

In [9]:
X

,HighBP,HighChol,CholCheck,BMI,HeartDiseaseorAttack,GenHlth,PhysHlth,DiffWalk,Age
0,1.0,0.0,1.0,0.162791,0.0,0.50,1.000000,0.0,0.250000
1,1.0,1.0,1.0,0.162791,0.0,0.50,0.000000,0.0,0.916667
2,0.0,0.0,1.0,0.162791,0.0,0.00,0.333333,0.0,1.000000
3,1.0,1.0,1.0,0.186047,0.0,0.50,0.100000,0.0,0.833333
4,0.0,0.0,1.0,0.197674,0.0,0.25,0.000000,0.0,0.583333
...,...,...,...,...,...,...,...,...,...
70687,0.0,1.0,1.0,0.290698,0.0,0.75,0.000000,0.0,0.416667
70688,0.0,1.0,1.0,0.197674,1.0,0.25,0.000000,1.0,0.750000
70689,1.0,1.0,1.0,0.151163,1.0,1.00,0.000000,1.0,1.000000
70690,1.0,1.0,1.0,0.069767,0.0,0.75,0.000000,1.0,0.833333


In [10]:
y

0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
70687    1.0
70688    1.0
70689    1.0
70690    1.0
70691    1.0
Name: Diabetes_binary, Length: 70692, dtype: float64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

#Decision Tree

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [13]:
#Test Case 1: We don't specify max depth
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.67      0.72      0.69      7090
         1.0       0.70      0.65      0.67      7049

    accuracy                           0.68     14139
   macro avg       0.68      0.68      0.68     14139
weighted avg       0.68      0.68      0.68     14139

Accuracy 0.6827215503218049


In [14]:
#Test Case 2: Max depth of 10
clf = DecisionTreeClassifier(max_depth=10)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.76      0.69      0.73      7090
         1.0       0.72      0.79      0.75      7049

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139

Accuracy 0.7385953744960747


In [15]:
#Test Case 3: Max depth of 15
clf = DecisionTreeClassifier(max_depth=15)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.72      0.70      0.71      7090
         1.0       0.71      0.73      0.72      7049

    accuracy                           0.71     14139
   macro avg       0.71      0.71      0.71     14139
weighted avg       0.71      0.71      0.71     14139

Accuracy 0.714689864912653


In [16]:
#Test Case 4: Max depth of 20
clf = DecisionTreeClassifier(max_depth=20)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.68      0.71      0.70      7090
         1.0       0.70      0.66      0.68      7049

    accuracy                           0.69     14139
   macro avg       0.69      0.69      0.69     14139
weighted avg       0.69      0.69      0.69     14139

Accuracy 0.6883089327392319


In [17]:
#Test Case 5: Max depth of 5
clf = DecisionTreeClassifier(max_depth=5)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.76      0.70      0.73      7090
         1.0       0.72      0.77      0.74      7049

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.73     14139
weighted avg       0.74      0.74      0.73     14139

Accuracy 0.73520050922979


In [18]:
#Test Case 6: Max depth of 8
clf = DecisionTreeClassifier(max_depth=8)
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.77      0.69      0.73      7090
         1.0       0.72      0.79      0.75      7049

    accuracy                           0.74     14139
   macro avg       0.74      0.74      0.74     14139
weighted avg       0.74      0.74      0.74     14139

Accuracy 0.7411415234457882


Best model is Test Case 6 with max depth of 8

#Naive Bayes

In [19]:
from sklearn.naive_bayes import *
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [20]:
#Test Case 1: Gaussian Naive Bayes
clf = GaussianNB()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy: ", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.74      0.70      0.72      7090
         1.0       0.72      0.75      0.73      7049

    accuracy                           0.73     14139
   macro avg       0.73      0.73      0.73     14139
weighted avg       0.73      0.73      0.73     14139

Accuracy:  0.7268547987835066


In [21]:
#Test Case 2: Multinomial Naive Bayes
clf = MultinomialNB()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy: ", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.63      0.77      0.69      7090
         1.0       0.70      0.53      0.61      7049

    accuracy                           0.65     14139
   macro avg       0.66      0.65      0.65     14139
weighted avg       0.66      0.65      0.65     14139

Accuracy:  0.6540773746375274


In [22]:
#Test Case 3: Complement Naive Bayes
clf = ComplementNB()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy: ", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.63      0.77      0.69      7090
         1.0       0.70      0.53      0.61      7049

    accuracy                           0.65     14139
   macro avg       0.66      0.65      0.65     14139
weighted avg       0.66      0.65      0.65     14139

Accuracy:  0.6539359219180989


In [23]:
#Test Case 4: Bernoulli Naive Bayes
clf = BernoulliNB()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy: ", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.71      0.70      0.71      7090
         1.0       0.70      0.71      0.71      7049

    accuracy                           0.71     14139
   macro avg       0.71      0.71      0.71     14139
weighted avg       0.71      0.71      0.71     14139

Accuracy:  0.708324492538369


In [24]:
#Test Case 5: Categorial Naive Bayes
clf = CategoricalNB()
clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
print(classification_report(y_test, predictions))
print("Accuracy: ", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

         0.0       0.71      0.70      0.71      7090
         1.0       0.70      0.71      0.71      7049

    accuracy                           0.71     14139
   macro avg       0.71      0.71      0.71     14139
weighted avg       0.71      0.71      0.71     14139

Accuracy:  0.7067685126246552


Best model is Test Case 1 with Gaussian NB